In [2]:
from __future__ import annotations
import numpy as np

In [ ]:
class Node:
    def __init__(
        self, left: Node = None, right: Node = None, feature_idx: int = None, value=None
    ):
        self.left = left
        self.right = right
        self.value = value
        self.feature_idx = feature_idx

    def is_leaf_node(self):
        return self.value != None

In [ ]:
class DecisionTree:
    def fit(self, X: np.array, Y: np.array):
        self.labels = np.unique(Y)

        self.grow_tree()
            

    def predict(self, x: np.array):
        pass

    def traverse(self, node: Node, x: np.array):
        pass
    
    def impurity(node: Node, data: np.array, feature_idx: int):
        n = len(data)
        
        unique_feats_under_feature_idx = np.unique(data[feature_idx])
        subset = data[:,feature_idx]
        g_i = 0
        for unique_feat in unique_feats_under_feature_idx:
            
            subsubset = np.array([row for row in subset if row[0]==unique_feat])
            n_sss = len(subsubset)

            unique_y_counts = np.bincount(subsubset[:,1])
            # gini impurity for sub-sub-set
            g_i_sss = 1 - sum([(unique_y_count/n_sss)**2 for unique_y_count in unique_y_counts])

            g_i += n_sss*g_i_sss
        
        return g_i

In [1]:
import numpy as np
x = np.array([0, 1, 1, 2, 2, 2])
np.bincount(x)

array([1, 2, 3], dtype=int64)